In [1]:

from pathlib import Path
import torch
import sys
sys.path.insert(0, "..")   
from src.data_utils import load_raw_data, load_full_data, load_X_y_data, create_sale_features,build_feature_and_label_cols
from src.model_utils import generate_loaders, dataloader_to_dataframe, combine_loaders_to_dataframe
from src.model import naive_percent_mav
import pandas as pd
from src.data_utils import (
    sort_df,
    build_feature_and_label_cols,
    get_X_feature_idx,
    get_y_idx,
    WEIGHT_COLUMN,
    META_FEATURES,
    UNIT_SALES,
    X_SALE_FEATURES,
    X_CYCLICAL_FEATURES,
    X_FEATURES,
    X_TO_LOG_FEATURES,
    X_LOG_FEATURES,
    LABELS,
    Y_LOG_FEATURES,
    Y_TO_LOG_FEATURES,
    ALL_FEATURES,
    generate_sales_features,
    generate_cyclical_features,
    create_features,
    create_cyc_features,
    create_sale_features,
    create_y_targets_from_shift,
    prepare_training_data_from_raw_df,
    sort_df,
    add_rolling_sales_summaries,
)


In [2]:
%reload_ext autoreload
%autoreload 2

In [10]:
df = pd.read_parquet("../output/data/train_2014_January_12_store_20_item.parquet")
cols = [
        "date",
        "store_item",
        "store",
        "item",
        "weight",
        "onpromotion",
        "unit_sales"
    ]
df = df[cols]
df['store_item'] = df['store_item'].astype(str)
df.sort_values(["store_item", "date"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,store_item,store,item,weight,onpromotion,unit_sales
0,2014-01-02,1_1047679,1,1047679,1,None,53.0
1,2014-01-03,1_1047679,1,1047679,1,None,50.0
2,2014-01-04,1_1047679,1,1047679,1,None,32.0
3,2014-01-05,1_1047679,1,1047679,1,None,11.0
4,2014-01-06,1_1047679,1,1047679,1,None,75.0


In [11]:
df.dtypes

date           datetime64[ns]
store_item             object
store                   int64
item                    int64
weight                  int64
onpromotion            object
unit_sales            float32
dtype: object

In [12]:
df.to_csv("../output/data/train_2014_January_12_store_20_item.csv", index=False)

In [9]:
df = pd.read_parquet("../output/data/train_2014_January_12_store_20_item_cluster.parquet")
cols = [
        "date",
        "store_item",
        "store",
        "item",
        "store_cluster",
        "item_cluster",
        "weight",
        "onpromotion",
        "unit_sales"
    ]
df = df[cols]
df['store_item'] = df['store_item'].astype(str)
df.sort_values(["date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,store_item,store,item,store_cluster,item_cluster,weight,onpromotion,unit_sales
0,2014-01-01,1_1047679,1,1047679,8,1,1,False,0.0
1,2014-01-01,1_1168718,1,1168718,8,0,1,False,0.0
2,2014-01-01,1_1463591,1,1463591,8,0,1,False,0.0
3,2014-01-01,1_1463992,1,1463992,8,4,1,False,0.0
4,2014-01-01,1_1464092,1,1464092,8,5,1,False,0.0


In [10]:
df.tail()

,date,store_item,store,item,store_cluster,item_cluster,weight,onpromotion,unit_sales
7435,2014-01-31,47_671076,47,671076,1,1,1,False,6.0
7436,2014-01-31,47_807493,47,807493,1,6,1,False,118.0
7437,2014-01-31,47_850542,47,850542,1,0,1,False,0.0
7438,2014-01-31,47_864508,47,864508,1,0,1,False,0.0
7439,2014-01-31,47_864510,47,864510,1,0,1,False,0.0


In [11]:
df.query("date == '2014-01-02'").head()

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_76447/3952204710.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df.query("date == '2014-01-02'").head()


,date,store_item,store,item,store_cluster,item_cluster,weight,onpromotion,unit_sales
240,2014-01-02,1_1047679,1,1047679,8,1,1,False,53.0
241,2014-01-02,1_1168718,1,1168718,8,0,1,False,1.0
242,2014-01-02,1_1463591,1,1463591,8,0,1,False,3.0
243,2014-01-02,1_1463992,1,1463992,8,4,1,False,100.0
244,2014-01-02,1_1464092,1,1464092,8,5,1,False,0.0


In [12]:
df.query("store_cluster =='8' & item_cluster == '1'")

,date,store_item,store,item,store_cluster,item_cluster,weight,onpromotion,unit_sales
0,2014-01-01,1_1047679,1,1047679,8,1,1,False,0.0
15,2014-01-01,1_671076,1,671076,8,1,1,False,0.0
240,2014-01-02,1_1047679,1,1047679,8,1,1,False,53.0
255,2014-01-02,1_671076,1,671076,8,1,1,False,0.0
480,2014-01-03,1_1047679,1,1047679,8,1,1,False,50.0
...,...,...,...,...,...,...,...,...,...
6735,2014-01-29,1_671076,1,671076,8,1,1,False,1.0
6960,2014-01-30,1_1047679,1,1047679,8,1,1,False,18.0
6975,2014-01-30,1_671076,1,671076,8,1,1,False,0.0
7200,2014-01-31,1_1047679,1,1047679,8,1,1,False,49.0


In [14]:
df.dtypes

date             datetime64[ns]
store_item               object
store                     int64
item                      int64
store_cluster            object
item_cluster             object
weight                    int64
onpromotion                bool
unit_sales              float32
dtype: object

In [ ]:
df2 = df.query("store == 1 & item == 1047679")

In [20]:
df2["store_cluster"].value_counts()

store_cluster
8    31
Name: count, dtype: int64

In [21]:
df2["item_cluster"].value_counts()

item_cluster
1    31
Name: count, dtype: int64

In [67]:
df = pd.read_parquet("../output/data/train_2014_January_12_store_20_item_cluster_medians.parquet")
cols = [
        "date",
        "store_item",
        "store",
        "item",
        "store_cluster",
        "store_cluster_median",
        "item_cluster",
        "item_cluster_median",
        "weight",
        "onpromotion",
        "unit_sales"
    ]
df = df[cols]
df['store_item'] = df['store_item'].astype(str)
df.sort_values(["date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,store_item,store,item,store_cluster,store_cluster_median,item_cluster,item_cluster_median,weight,onpromotion,unit_sales
0,2014-01-01,1_1047679,1,1047679,8,0.0,1,0.0,1,False,0.0
1,2014-01-01,1_1168718,1,1168718,8,0.0,0,0.0,1,False,0.0
2,2014-01-01,1_1463591,1,1463591,8,0.0,0,0.0,1,False,0.0
3,2014-01-01,1_1463992,1,1463992,8,0.0,4,0.0,1,False,0.0
4,2014-01-01,1_1464092,1,1464092,8,0.0,5,0.0,1,False,0.0


In [68]:
df.query("store_cluster =='8' & item_cluster == '1'")

,date,store_item,store,item,store_cluster,store_cluster_median,item_cluster,item_cluster_median,weight,onpromotion,unit_sales
0,2014-01-01,1_1047679,1,1047679,8,0.0,1,0.0,1,False,0.0
15,2014-01-01,1_671076,1,671076,8,0.0,1,0.0,1,False,0.0
240,2014-01-02,1_1047679,1,1047679,8,0.5,1,6.5,1,False,53.0
255,2014-01-02,1_671076,1,671076,8,0.5,1,6.5,1,False,0.0
480,2014-01-03,1_1047679,1,1047679,8,0.0,1,5.5,1,False,50.0
...,...,...,...,...,...,...,...,...,...,...,...
6735,2014-01-29,1_671076,1,671076,8,0.0,1,4.0,1,False,1.0
6960,2014-01-30,1_1047679,1,1047679,8,0.0,1,3.0,1,False,18.0
6975,2014-01-30,1_671076,1,671076,8,0.0,1,3.0,1,False,0.0
7200,2014-01-31,1_1047679,1,1047679,8,0.5,1,3.0,1,False,49.0


In [ ]:
df = pd.read_parquet("../output/data/sale_data_12_store_20_item/sale_features_cluster_8_1.parquet")
cols = [
        "start_date",
        "store_item",
        "store",
        "item",
        "store_cluster",
        "store_med_day_1",
        "item_cluster",
        "item_med_day_1",
        "weight",
        "sales_day_1"
    ]
df = df[cols]
df['store_item'] = df['store_item'].astype(str)
df.sort_values(["start_date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,start_date,store_item,store,item,store_cluster,store_med_day_1,item_cluster,item_med_day_1,weight,sales_day_1
0,2014-01-01,1_1047679,1,1047679,8,0.0,1,0.0,1,0.0
1,2014-01-01,1_671076,1,671076,8,0.0,1,0.0,1,0.0
2,2014-01-02,1_1047679,1,1047679,8,26.5,1,26.5,1,53.0
3,2014-01-02,1_671076,1,671076,8,26.5,1,26.5,1,0.0
4,2014-01-03,1_1047679,1,1047679,8,25.0,1,25.0,1,50.0


In [15]:
df = pd.read_parquet("../output/data/clustered_data_12_store_20_item/cluster_8_1.parquet")
cols = [
        "date",
        "store_item",
        "store",
        "item",
        "store_cluster",
        "store_cluster_median",
        "item_cluster",
        "item_cluster_median",
        "weight",
        "unit_sales"
    ]
df = df[cols]
df['store_item'] = df['store_item'].astype(str)
df.sort_values(["date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)


,date,store_item,store,item,store_cluster,store_cluster_median,item_cluster,item_cluster_median,weight,unit_sales
0,2014-01-01,1_1047679,1,1047679,8,0.0,1,0.0,1,0.0
1,2014-01-01,1_671076,1,671076,8,0.0,1,0.0,1,0.0
2,2014-01-02,1_1047679,1,1047679,8,0.5,1,6.5,1,53.0
3,2014-01-02,1_671076,1,671076,8,0.5,1,6.5,1,0.0
4,2014-01-03,1_1047679,1,1047679,8,0.0,1,5.5,1,50.0
5,2014-01-03,1_671076,1,671076,8,0.0,1,5.5,1,0.0
6,2014-01-04,1_1047679,1,1047679,8,0.5,1,8.5,1,32.0
7,2014-01-04,1_671076,1,671076,8,0.5,1,8.5,1,1.0
8,2014-01-05,1_1047679,1,1047679,8,0.0,1,7.0,1,11.0
9,2014-01-05,1_671076,1,671076,8,0.0,1,7.0,1,0.0


In [16]:
df2 = generate_sales_features(df, window_size=1, calendar_aligned=True, log_level="DEBUG")

2025-08-11 16:45:36,695 - INFO - Total rows: 62
2025-08-11 16:45:36,707 - DEBUG - Unique store_clusters: ['8']
2025-08-11 16:45:36,708 - DEBUG - Unique item_clusters: ['1']
2025-08-11 16:45:36,729 - DEBUG - Store vs. Item cluster crosstab:
item_cluster    1
store_cluster    
8              62
2025-08-11 16:45:36,730 - DEBUG - Generating rolling windows
2025-08-11 16:45:36,744 - INFO - Processing window: 2014-01-01 00:00:00 to 2014-01-01 00:00:00
Window 2014-01-01: 100%|██████████| 2/2 [00:00<00:00, 240.47it/s]
2025-08-11 16:45:36,781 - INFO - Processing window: 2014-01-02 00:00:00 to 2014-01-02 00:00:00
Window 2014-01-02: 100%|██████████| 2/2 [00:00<00:00, 687.31it/s]
2025-08-11 16:45:36,805 - INFO - Processing window: 2014-01-03 00:00:00 to 2014-01-03 00:00:00
Window 2014-01-03: 100%|██████████| 2/2 [00:00<00:00, 498.37it/s]
2025-08-11 16:45:36,831 - INFO - Processing window: 2014-01-04 00:00:00 to 2014-01-04 00:00:00
Window 2014-01-04: 100%|██████████| 2/2 [00:00<00:00, 683.50it/s]
2

In [17]:
df2.head()

,start_date,store_item,store,item,store_cluster,item_cluster,weight,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,store_med_logpct_change_1,item_med_logpct_change_1,sales_day_1
0,2014-01-01,1_671076,1,671076,8,1,1,0.0,0.0,0.0,0.000000,-6.907755,-6.907755,0.0
1,2014-01-01,1_1047679,1,1047679,8,1,1,0.0,0.0,0.0,0.000000,-6.907755,-6.907755,0.0
2,2014-01-02,1_671076,1,671076,8,1,1,0.5,6.5,0.0,0.000000,-6.907755,-6.907755,0.0
3,2014-01-02,1_1047679,1,1047679,8,1,1,0.5,6.5,106.0,8.153846,4.663439,2.098490,53.0
4,2014-01-03,1_671076,1,671076,8,1,1,0.0,5.5,0.0,0.000000,-6.907755,-6.907755,0.0


In [19]:
# 1) Build long-DF rollups
long_with_rolls = add_rolling_sales_summaries(
    df, windows=(3,7,14,30,60)
)

# 2) Figure out which columns are new
from typing import Iterable
def new_cols(df_new, df_old) -> list[str]:
    return [c for c in df_new.columns if c not in df_old.columns]

roll_cols = new_cols(long_with_rolls, df2)

# 3) Merge onto your existing features
keys = ["store_item", "start_date"]
features = df.merge(
    long_with_rolls[keys + roll_cols],
    on=keys, how="left", validate="one_to_one"  # or "many_to_one" if needed
)


KeyError: "Missing required columns: ['start_date']"

In [8]:
df2.columns

Index(['start_date', 'store_item', 'store', 'item', 'store_cluster',
       'item_cluster', 'weight', 'store_med_day_1', 'item_med_day_1',
       'store_med_change_1', 'item_med_change_1', 'store_med_logpct_change_1',
       'item_med_logpct_change_1', 'sales_day_1'],
      dtype='object')

In [7]:
df2.head(10)

,start_date,store_item,store,item,store_cluster,item_cluster,weight,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,store_med_logpct_change_1,item_med_logpct_change_1,sales_day_1
0,2014-01-01,1_1047679,1,1047679,8,1,1,0.0,0.0,0.0,0.000000,-6.907755,-6.907755,0.0
1,2014-01-02,1_1047679,1,1047679,8,1,1,0.5,6.5,106.0,8.153846,4.663439,2.098490,53.0
2,2014-01-03,1_1047679,1,1047679,8,1,1,0.0,5.5,50000.0,9.090909,10.819778,2.207275,50.0
3,2014-01-04,1_1047679,1,1047679,8,1,1,0.5,8.5,64.0,3.764706,4.158883,1.325670,32.0
4,2014-01-05,1_1047679,1,1047679,8,1,1,0.0,7.0,11000.0,1.571429,9.305651,0.451985,11.0
5,2014-01-06,1_1047679,1,1047679,8,1,1,1.0,7.0,75.0,10.714286,4.317488,2.371578,75.0
6,2014-01-07,1_1047679,1,1047679,8,1,1,0.5,1.0,68.0,34.000000,4.219508,3.526361,34.0
7,2014-01-08,1_1047679,1,1047679,8,1,1,0.0,4.0,14000.0,3.500000,9.546813,1.252763,14.0
8,2014-01-09,1_1047679,1,1047679,8,1,1,0.0,3.5,26000.0,7.428571,10.165852,2.005333,26.0
9,2014-01-10,1_1047679,1,1047679,8,1,1,0.0,6.5,24000.0,3.692308,10.085809,1.306252,24.0


In [85]:
df = pd.read_parquet("../output/data/sale_data_12_store_20_item/sale_features_cluster_8_1.parquet")
df.sort_values(["start_date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,start_date,store_item,store,item,store_cluster,item_cluster,weight,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,store_med_logpct_change_1,item_med_logpct_change_1,sales_day_1
0,2014-01-01,1_1047679,1,1047679,8,1,1,0.0,0.0,0.0,0.000000,-6.907755,-6.907755,0.0
1,2014-01-01,1_671076,1,671076,8,1,1,0.0,0.0,0.0,0.000000,-6.907755,-6.907755,0.0
2,2014-01-02,1_1047679,1,1047679,8,1,1,0.5,6.5,106.0,8.153846,4.663439,2.098490,53.0
3,2014-01-02,1_671076,1,671076,8,1,1,0.5,6.5,0.0,0.000000,-6.907755,-6.907755,0.0
4,2014-01-03,1_1047679,1,1047679,8,1,1,0.0,5.5,50000.0,9.090909,10.819778,2.207275,50.0
5,2014-01-03,1_671076,1,671076,8,1,1,0.0,5.5,0.0,0.000000,-6.907755,-6.907755,0.0
6,2014-01-04,1_1047679,1,1047679,8,1,1,0.5,8.5,64.0,3.764706,4.158883,1.325670,32.0
7,2014-01-04,1_671076,1,671076,8,1,1,0.5,8.5,2.0,0.117647,0.693147,-2.140066,1.0
8,2014-01-05,1_1047679,1,1047679,8,1,1,0.0,7.0,11000.0,1.571429,9.305651,0.451985,11.0
9,2014-01-05,1_671076,1,671076,8,1,1,0.0,7.0,0.0,0.000000,-6.907755,-6.907755,0.0


In [60]:
df = pd.read_parquet("../output/data/sale_cyc_data_2014_2015_top_53_store_2000_item/sale_cyc_features_17_15.parquet")
df.sort_values(["start_date", "store_item"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1,store_med_logpct_change_1,...,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1
0,2014-01-01,12_1012473,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
1,2014-01-01,12_1167614,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
2,2014-01-01,12_1464238,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
3,2014-01-01,12_1489837,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
4,2014-01-01,12_155500,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
5,2014-01-01,12_1686637,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
6,2014-01-01,12_264305,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
7,2014-01-01,12_584126,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
8,2014-01-01,12_587157,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116
9,2014-01-01,12_638977,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.9741,0.226116


In [58]:
df.columns

Index(['start_date', 'store_item', 'store_cluster', 'item_cluster',
       'store_med_day_1', 'item_med_day_1', 'store_med_change_1',
       'item_med_change_1', 'sales_day_1', 'store_med_logpct_change_1',
       'item_med_logpct_change_1', 'dayofweek_sin_1', 'dayofweek_cos_1',
       'weekofmonth_sin_1', 'weekofmonth_cos_1', 'monthofyear_sin_1',
       'monthofyear_cos_1', 'paycycle_sin_1', 'paycycle_cos_1', 'season_sin_1',
       'season_cos_1'],
      dtype='object')

In [8]:
df.query("sales_day_1 == 0")

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1,store_med_logpct_change_1,...,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1
0,2014-01-01,12_1012473,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,9.510565e-01,0.309017,5.000000e-01,0.866025,0.406737,0.913545,-0.974100,0.226116
1,2014-01-01,12_1167614,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,9.510565e-01,0.309017,5.000000e-01,0.866025,0.406737,0.913545,-0.974100,0.226116
2,2014-01-01,12_1464238,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,9.510565e-01,0.309017,5.000000e-01,0.866025,0.406737,0.913545,-0.974100,0.226116
3,2014-01-01,12_1489837,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,9.510565e-01,0.309017,5.000000e-01,0.866025,0.406737,0.913545,-0.974100,0.226116
4,2014-01-01,12_155500,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,9.510565e-01,0.309017,5.000000e-01,0.866025,0.406737,0.913545,-0.974100,0.226116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23990,2015-12-30,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.974928,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.000000,1.000000,-0.981306,0.192452
23991,2015-12-31,12_1012473,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.433884,-0.900969,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.000000,1.000000,-0.977848,0.209315
24014,2015-12-31,14_1167614,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.433884,-0.900969,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.000000,1.000000,-0.977848,0.209315
24022,2015-12-31,14_638977,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.433884,-0.900969,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.000000,1.000000,-0.977848,0.209315


In [42]:
df.to_csv("../output/data/sale_cyc_features_17_15.csv", index=False)

In [30]:
df[['start_date', 'store_item', 'store_cluster', 'item_cluster','sales_day_1', 'store_med_logpct_change_1',
       'item_med_logpct_change_1']]

,start_date,store_item,store_cluster,item_cluster,sales_day_1,store_med_logpct_change_1,item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.000000,-6.907755,-6.907755
1,2014-01-01,12_1167614,17,15,0.000000,-6.907755,-6.907755
2,2014-01-01,12_1464238,17,15,0.000000,-6.907755,-6.907755
3,2014-01-01,12_1489837,17,15,0.000000,-6.907755,-6.907755
4,2014-01-01,12_155500,17,15,0.000000,-6.907755,-6.907755
...,...,...,...,...,...,...,...
24019,2015-12-31,14_264305,17,15,25.000000,10.126631,10.126631
24020,2015-12-31,14_584126,17,15,133.658005,11.803040,11.803040
24021,2015-12-31,14_587157,17,15,113.089996,11.635939,11.635939
24022,2015-12-31,14_638977,17,15,0.000000,-6.907755,-6.907755


In [32]:
df['sales_day_1'].value_counts()

sales_day_1
0.000000      5091
10.000000      630
12.000000      627
13.000000      575
16.000000      551
              ... 
21.445999        1
156.373993       1
21.996000        1
85.029999        1
113.089996       1
Name: count, Length: 4256, dtype: int64

In [61]:
df = pd.read_parquet("../output/data/sale_cyc_features_X_1_day_y_2014_2015_top_53_store_2000_item/sale_cyc_features_X_1_day_y_17_15.parquet")
df = sort_df(df, window_size=1)
df.head()

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1,store_med_logpct_change_1,...,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,0.5,0.866025,0.406737,0.913545,-0.974100,0.226116,0.0,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,0.5,0.866025,0.743145,0.669131,-0.970064,0.242850,38.0,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,0.0,0.0,38000.0,38000.0,38.0,10.545341,...,0.309017,0.5,0.866025,0.951057,0.309017,-0.965740,0.259512,12.0,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,0.0,0.0,12000.0,12000.0,12.0,9.392662,...,0.309017,0.5,0.866025,0.994522,-0.104529,-0.961130,0.276097,18.0,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,0.0,0.0,18000.0,18000.0,18.0,9.798127,...,0.309017,0.5,0.866025,0.866025,-0.500000,-0.956235,0.292600,20.0,9.903488,9.903488


In [33]:
df.tail()

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1,store_med_logpct_change_1,...,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
23920,2015-12-26,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,-2.449294e-16,1.0,0.0,1.0,-0.992222,0.124479,0.0,-6.907755,-6.907755
23921,2015-12-27,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,-2.449294e-16,1.0,0.0,1.0,-0.989932,0.141540,0.0,-6.907755,-6.907755
23922,2015-12-28,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,-2.449294e-16,1.0,0.0,1.0,-0.987349,0.158560,0.0,-6.907755,-6.907755
23923,2015-12-29,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,1.000000,-2.449294e-16,1.0,0.0,1.0,-0.984474,0.175532,0.0,-6.907755,-6.907755
23924,2015-12-30,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,1.000000,-2.449294e-16,1.0,0.0,1.0,-0.981306,0.192452,0.0,-6.907755,-6.907755


In [21]:
df.to_csv("../output/data/sale_cyc_features_X_1_day_y_17_15.csv", index=False)

In [18]:
df_y = df[features[LABELS]]
summary = df_y.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
summary

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
y_sales_day_1,23925.0,28.913504,47.886642,0.000000,0.000000,0.000000,4.00000,15.000000,30.000000,113.896402,208.955121,3001.000000
y_store_med_logpct_change_1,23925.0,6.402333,6.949534,-6.907755,-6.907755,-6.907755,8.29405,9.615805,10.308953,11.643045,12.249875,14.914456
y_item_med_logpct_change_1,23925.0,6.402333,6.949534,-6.907755,-6.907755,-6.907755,8.29405,9.615805,10.308953,11.643045,12.249875,14.914456


In [19]:
df.columns

Index(['start_date', 'store_item', 'store_cluster', 'item_cluster',
       'store_med_day_1', 'item_med_day_1', 'store_med_change_1',
       'item_med_change_1', 'sales_day_1', 'store_med_logpct_change_1',
       'item_med_logpct_change_1', 'dayofweek_sin_1', 'dayofweek_cos_1',
       'weekofmonth_sin_1', 'weekofmonth_cos_1', 'monthofyear_sin_1',
       'monthofyear_cos_1', 'paycycle_sin_1', 'paycycle_cos_1', 'season_sin_1',
       'season_cos_1', 'y_sales_day_1', 'y_store_med_logpct_change_1',
       'y_item_med_logpct_change_1'],
      dtype='object')

In [6]:
#df.to_csv("../output/data/sale_cyc_features_X_1_day_y_17_15.csv", index=False)

In [7]:
%pwd

'/Users/yvesgreatti/github/kaggle_favorita_grocery_sales_forecasting/notebook'

In [8]:
train_loader = torch.load( Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/") / "17_15_train_loader.pt", weights_only=False)

In [9]:
xb, yb, wb = next(iter(train_loader))
print("yb (raw targets) min:", yb.min().item(), "max:", yb.max().item())
print("Unique targets:", torch.unique(yb))


yb (raw targets) min: -6.907755374908447 max: 10.933107376098633
Unique targets: tensor([-6.9078,  0.0000,  1.6094,  2.5649,  2.6391,  2.9444,  2.9957,  3.0445,
         3.1781,  3.2189,  3.2581,  3.2958,  3.3322,  3.4657,  3.4965,  3.6109,
         3.6376,  3.6636,  3.7377,  3.8286,  3.8501,  4.0431,  8.2940,  9.3927,
         9.4727,  9.7981,  9.8522,  9.9035, 10.0432, 10.0858, 10.1266, 10.1659,
        10.2036, 10.3417, 10.3735, 10.4913, 10.5187, 10.5453, 10.6213, 10.7144,
        10.7364, 10.9331])


In [10]:
import torch

def scan_loader_for_nans(loader, max_batches=5):
    for batch_idx, (xb, yb, wb) in enumerate(loader):
        if torch.isnan(xb).any():
            print(f"NaN in X at batch {batch_idx}")
        if torch.isnan(yb).any():
            print(f"NaN in Y at batch {batch_idx}")
        if torch.isnan(wb).any():
            print(f"NaN in W at batch {batch_idx}")
        if torch.isinf(xb).any():
            print(f"Inf in X at batch {batch_idx}")
        if torch.isinf(yb).any():
            print(f"Inf in Y at batch {batch_idx}")
        if torch.isinf(wb).any():
            print(f"Inf in W at batch {batch_idx}")
        if batch_idx + 1 >= max_batches:
            break

scan_loader_for_nans(train_loader)


In [11]:
X, Y, W = train_loader.dataset.tensors

In [12]:
X.shape

torch.Size([23895, 17])

In [13]:
Y[:,0]

tensor([0.0000, 3.6636, 2.5649,  ..., 0.0000, 0.0000, 0.0000])

In [6]:
features = build_feature_and_label_cols(1)

In [16]:
df_y = pd.DataFrame(Y.numpy(), columns=features[LABELS])
summary = df_y.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
summary


,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
y_sales_day_1,23895.0,2.474072,1.538100,0.000000,0.000000,0.000000,1.609438,2.772589,3.433987,4.744255,5.347145,8.007034
y_store_med_logpct_change_1,23895.0,6.419044,6.937863,-6.907755,-6.907755,-6.907755,8.294049,9.615806,10.308952,11.643271,12.250128,14.914456
y_item_med_logpct_change_1,23895.0,6.419044,6.937863,-6.907755,-6.907755,-6.907755,8.294049,9.615806,10.308952,11.643271,12.250128,14.914456


In [17]:
df_w = pd.DataFrame(W.numpy(), columns=["weight"])
summary = df_w.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
summary

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
weight,23895.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [57]:
val_loader = torch.load( Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/") / "17_15_val_loader.pt", weights_only=False)
xb, yb, wb = next(iter(val_loader))
y_sales = torch.expm1(yb[:, get_y_idx(1)[Y_TO_LOG_FEATURES]])
print("Val batch: min =", y_sales.min().item(), "max =", y_sales.max().item())

Val batch: min = 0.0 max = 114.99999237060547


In [58]:
y_sales_all = torch.expm1(yb[:, get_y_idx(1)[Y_TO_LOG_FEATURES]])
row_all_zero = (y_sales_all <= 0).all(dim=1)
print("Rows all zeros:", row_all_zero.sum().item(), "/", y_sales_all.shape[0])


Rows all zeros: 3 / 32


In [59]:
empty_batches = 0
for i, (_, yb, _) in enumerate(val_loader):
    y_sales_all = torch.expm1(yb[:, get_y_idx(1)[Y_TO_LOG_FEATURES]])
    if (y_sales_all <= 0).all():
        empty_batches += 1
        print(f"Empty batch #{i}")
print(f"Total empty batches: {empty_batches} / {len(val_loader)}")


Empty batch #28
Total empty batches: 1 / 29


In [42]:
yb[:,0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [43]:
df2 = dataloader_to_dataframe(
    loader_path=Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_val_loader.pt"),
    meta_path= Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_val_meta.parquet"),
    scaler_dir= Path("../output/data/scalers_2014_2015_top_53_store_2000_item/"),
    window_size=1,
)

In [44]:
df2.head()

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1,store_med_logpct_change_1,...,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2015-11-29,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,1.000000,-0.5,0.866025,0.000000,1.000000,-0.942761,-0.333469,0.0,-6.907755,-6.907755
1,2015-11-30,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,1.000000,-0.5,0.866025,0.000000,1.000000,-0.948362,-0.317191,0.0,-6.907755,-6.907755
2,2015-12-01,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,0.0,1.000000,0.406737,0.913545,-0.953681,-0.300820,0.0,-6.907755,-6.907755
3,2015-12-02,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,0.0,1.000000,0.743145,0.669130,-0.958718,-0.284359,0.0,-6.907755,-6.907755
4,2015-12-03,14_770449,17,15,0.0,0.0,0.0,0.0,0.0,-6.907755,...,0.309017,0.0,1.000000,0.951057,0.309017,-0.963471,-0.267814,0.0,-6.907755,-6.907755


In [45]:
df2.columns

Index(['start_date', 'store_item', 'store_cluster', 'item_cluster',
       'store_med_day_1', 'item_med_day_1', 'store_med_change_1',
       'item_med_change_1', 'sales_day_1', 'store_med_logpct_change_1',
       'item_med_logpct_change_1', 'dayofweek_sin_1', 'dayofweek_cos_1',
       'weekofmonth_sin_1', 'weekofmonth_cos_1', 'monthofyear_sin_1',
       'monthofyear_cos_1', 'paycycle_sin_1', 'paycycle_cos_1', 'season_sin_1',
       'season_cos_1', 'y_sales_day_1', 'y_store_med_logpct_change_1',
       'y_item_med_logpct_change_1'],
      dtype='object')

In [46]:
df2["store_item"].nunique()

1

In [ ]:
df2

In [12]:
naive_percent_mav(df2)

0.0

In [39]:
df3 = df[features[META_FEATURES] + features[LABELS]]
df3.query("store_item == '12_1012473'")

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.0,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,38.0,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,12.0,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,18.0,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,20.0,9.903488,9.903488
...,...,...,...,...,...,...,...
720,2015-12-26,12_1012473,17,15,10.0,9.210340,9.210340
721,2015-12-27,12_1012473,17,15,30.0,10.308953,10.308953
722,2015-12-28,12_1012473,17,15,12.0,9.392662,9.392662
723,2015-12-29,12_1012473,17,15,42.0,10.645425,10.645425


In [40]:
df3 = df2[features[META_FEATURES] + features[LABELS]]
df3.query("store_item == '12_1012473'")

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.000000,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,37.999996,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,11.999999,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,18.000000,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,20.000002,9.903487,9.903487
...,...,...,...,...,...,...,...
720,2015-12-26,12_1012473,17,15,10.000001,9.210340,9.210340
721,2015-12-27,12_1012473,17,15,29.999998,10.308952,10.308952
722,2015-12-28,12_1012473,17,15,11.999999,9.392662,9.392662
723,2015-12-29,12_1012473,17,15,42.000004,10.645425,10.645425


In [16]:
df[features[META_FEATURES] + features[LABELS]].tail(10)

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
23915,2015-12-19,14_770449,17,15,0.0,-6.907755,-6.907755
23916,2015-12-20,14_770449,17,15,0.0,-6.907755,-6.907755
23917,2015-12-21,14_770449,17,15,0.0,-6.907755,-6.907755
23918,2015-12-22,14_770449,17,15,0.0,-6.907755,-6.907755
23919,2015-12-23,14_770449,17,15,0.0,-6.907755,-6.907755
23920,2015-12-26,14_770449,17,15,0.0,-6.907755,-6.907755
23921,2015-12-27,14_770449,17,15,0.0,-6.907755,-6.907755
23922,2015-12-28,14_770449,17,15,0.0,-6.907755,-6.907755
23923,2015-12-29,14_770449,17,15,0.0,-6.907755,-6.907755
23924,2015-12-30,14_770449,17,15,0.0,-6.907755,-6.907755


In [22]:
df_y = df2[features[LABELS]]
summary = df_y.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
summary

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
y_sales_day_1,30.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
y_store_med_logpct_change_1,30.0,-6.907756,0.0,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755
y_item_med_logpct_change_1,30.0,-6.907756,0.0,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755


In [62]:
generate_loaders(
    df,
    window_size=1,
    dataloader_dir=Path("../output/data/dataloader_2014_2015_top_53_store_2000_item"),
    scalers_dir=Path("../output/data/scalers_2014_2015_top_53_store_2000_item"),
    log_level="DEBUG",
)

2025-08-14 18:08:16,965 - INFO - Unique store items: 33
2025-08-14 18:08:16,967 - INFO - Preparing loaders for cluster 17_15 with 23925 rows
2025-08-14 18:08:16,975 - INFO - Global validation cutoff date: 2015-12-01
2025-08-14 18:08:16,977 - INFO - Train rows: 23001, Val rows: 924
2025-08-14 18:08:17,011 - INFO - [Sanity] X_TO_LOG_FEATURES scaled range: min=0.0000, max=1.0000
2025-08-14 18:08:17,013 - INFO - [Sanity] X_LOG_FEATURES scaled range: min=0.0000, max=1.0000
2025-08-14 18:08:17,016 - INFO - [Sanity] X_CYCLICAL_FEATURES scaled range: min=0.0000, max=1.0000
2025-08-14 18:08:17,083 - INFO - Saved scaler x_log1p for 17_15 to ../output/data/scalers_2014_2015_top_53_store_2000_item/17_15_x_log1p_scaler.pkl
2025-08-14 18:08:17,085 - INFO - Saved scaler x_log_raw for 17_15 to ../output/data/scalers_2014_2015_top_53_store_2000_item/17_15_x_log_raw_scaler.pkl
2025-08-14 18:08:17,086 - INFO - Saved scaler x_cyc for 17_15 to ../output/data/scalers_2014_2015_top_53_store_2000_item/17_15_x

(<torch.utils.data.dataloader.DataLoader at 0x16ec6bf50>,
 <torch.utils.data.dataloader.DataLoader at 0x16f7506e0>)

In [52]:
df2 = dataloader_to_dataframe(
    loader_path=Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_train_loader.pt"),
    meta_path= Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_train_meta.parquet"),
    scaler_dir= Path("../output/data/scalers_2014_2015_top_53_store_2000_item/"),
    window_size=1,
)

In [53]:
df2['store_item'].nunique()

33

In [54]:
df3 = df2[features[META_FEATURES] + features[LABELS]]
df3.query("store_item == '12_1012473'")

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.000000,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,37.999996,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,11.999999,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,18.000000,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,20.000002,9.903487,9.903487
...,...,...,...,...,...,...,...
692,2015-11-26,12_1012473,17,15,14.000001,9.546813,9.546813
693,2015-11-27,12_1012473,17,15,6.000000,8.699514,8.699514
694,2015-11-28,12_1012473,17,15,5.000000,8.517193,8.517193
695,2015-11-29,12_1012473,17,15,29.000002,10.275051,10.275051


In [55]:
df2 = dataloader_to_dataframe(
    loader_path=Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_val_loader.pt"),
    meta_path= Path("../output/data/dataloader_2014_2015_top_53_store_2000_item/17_15_val_meta.parquet"),
    scaler_dir= Path("../output/data/scalers_2014_2015_top_53_store_2000_item/"),
    window_size=1,
)

In [56]:
df3 = df2[features[META_FEATURES] + features[LABELS]]
df3.query("store_item == '12_1012473'")

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2015-12-01,12_1012473,17,15,24.999998,10.126631,10.126631
1,2015-12-02,12_1012473,17,15,11.999999,9.392662,9.392662
2,2015-12-03,12_1012473,17,15,60.999996,11.018629,11.018629
3,2015-12-04,12_1012473,17,15,5.000000,8.517193,8.517193
4,2015-12-05,12_1012473,17,15,17.000000,9.740969,9.740969
5,2015-12-06,12_1012473,17,15,33.000000,10.404263,10.404263
6,2015-12-07,12_1012473,17,15,33.000000,10.404263,10.404263
7,2015-12-08,12_1012473,17,15,9.000000,9.104980,9.104980
8,2015-12-09,12_1012473,17,15,0.000000,-6.907755,-6.907755
9,2015-12-10,12_1012473,17,15,16.000000,9.680344,9.680344


In [94]:
df3 = df[features[META_FEATURES] + features[X_TO_LOG_FEATURES]]
df3.query("store_item == '14_770449'")

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1
23200,2014-01-01,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
23201,2014-01-02,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
23202,2014-01-03,14_770449,17,15,0.0,0.0,5000.0,5000.0,5.0
23203,2014-01-04,14_770449,17,15,0.0,0.0,50000.0,50000.0,50.0
23204,2014-01-05,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23920,2015-12-26,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
23921,2015-12-27,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
23922,2015-12-28,14_770449,17,15,0.0,0.0,0.0,0.0,0.0
23923,2015-12-29,14_770449,17,15,0.0,0.0,0.0,0.0,0.0


In [95]:
df[features["UNIT_SALES"]].value_counts()

sales_day_1
0.000000       5082
10.000000       629
12.000000       623
13.000000       574
16.000000       550
               ... 
35.247002         1
35.342999         1
35.428001         1
35.557999         1
3001.000000       1
Name: count, Length: 4240, dtype: int64

In [96]:
df3 = df2[features[META_FEATURES] + features[X_TO_LOG_FEATURES]]
df3.query("store_item == '14_770449'")

,start_date,store_item,store_cluster,item_cluster,store_med_day_1,item_med_day_1,store_med_change_1,item_med_change_1,sales_day_1
23200,2014-01-01,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
23201,2014-01-02,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
23202,2014-01-03,14_770449,17,15,0.0,0.0,4999.999512,4999.999512,5.0
23203,2014-01-04,14_770449,17,15,0.0,0.0,50000.007812,50000.007812,50.0
23204,2014-01-05,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
23890,2015-11-24,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
23891,2015-11-25,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
23892,2015-11-26,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0
23893,2015-11-27,14_770449,17,15,0.0,0.0,0.000000,0.000000,0.0


In [97]:
df2[features["UNIT_SALES"]].value_counts()

sales_day_1
0.000000       5052
10.000001       629
11.999999       623
13.000001       574
16.000000       550
               ... 
35.246998         1
35.342999         1
35.427998         1
35.558002         1
3001.000977       1
Name: count, Length: 4240, dtype: int64

In [98]:
df3 = df[features[META_FEATURES] + features[X_LOG_FEATURES]]
df3.query("store_item == '12_1012473'")



,start_date,store_item,store_cluster,item_cluster,store_med_logpct_change_1,item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,-6.907755,-6.907755
2,2014-01-03,12_1012473,17,15,10.545341,10.545341
3,2014-01-04,12_1012473,17,15,9.392662,9.392662
4,2014-01-05,12_1012473,17,15,9.798127,9.798127
...,...,...,...,...,...,...
720,2015-12-26,12_1012473,17,15,10.621327,10.621327
721,2015-12-27,12_1012473,17,15,9.210340,9.210340
722,2015-12-28,12_1012473,17,15,10.308953,10.308953
723,2015-12-29,12_1012473,17,15,9.392662,9.392662


In [99]:
df3 = df2[features[META_FEATURES] + features[X_LOG_FEATURES]]
df3.query("store_item == '12_1012473'")


,start_date,store_item,store_cluster,item_cluster,store_med_logpct_change_1,item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,-6.907755,-6.907755
2,2014-01-03,12_1012473,17,15,10.545341,10.545341
3,2014-01-04,12_1012473,17,15,9.392662,9.392662
4,2014-01-05,12_1012473,17,15,9.798128,9.798128
...,...,...,...,...,...,...
720,2015-12-26,12_1012473,17,15,10.621327,10.621327
721,2015-12-27,12_1012473,17,15,9.210340,9.210340
722,2015-12-28,12_1012473,17,15,10.308951,10.308951
723,2015-12-29,12_1012473,17,15,9.392662,9.392662


In [100]:
df[features[META_FEATURES] + features[X_CYCLICAL_FEATURES]].head(10)

,start_date,store_item,store_cluster,item_cluster,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1
0,2014-01-01,12_1012473,17,15,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.974100,0.226116
1,2014-01-02,12_1012473,17,15,0.433884,-0.900969,0.951057,0.309017,0.5,0.866025,0.743145,0.669131,-0.970064,0.242850
2,2014-01-03,12_1012473,17,15,-0.433884,-0.900969,0.951057,0.309017,0.5,0.866025,0.951057,0.309017,-0.965740,0.259512
3,2014-01-04,12_1012473,17,15,-0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.994522,-0.104529,-0.961130,0.276097
4,2014-01-05,12_1012473,17,15,-0.781832,0.623490,0.951057,0.309017,0.5,0.866025,0.866025,-0.500000,-0.956235,0.292600
5,2014-01-06,12_1012473,17,15,0.000000,1.000000,0.951057,0.309017,0.5,0.866025,0.587785,-0.809017,-0.951056,0.309017
6,2014-01-07,12_1012473,17,15,0.781832,0.623490,0.951057,0.309017,0.5,0.866025,0.207912,-0.978148,-0.945596,0.325342
7,2014-01-08,12_1012473,17,15,0.974928,-0.222521,0.587785,-0.809017,0.5,0.866025,-0.207912,-0.978148,-0.939856,0.341571
8,2014-01-09,12_1012473,17,15,0.433884,-0.900969,0.587785,-0.809017,0.5,0.866025,-0.587785,-0.809017,-0.933837,0.357699
9,2014-01-10,12_1012473,17,15,-0.433884,-0.900969,0.587785,-0.809017,0.5,0.866025,-0.866025,-0.500000,-0.927542,0.373720


In [101]:
df2[features[META_FEATURES] + features[X_CYCLICAL_FEATURES]].head(10)

,start_date,store_item,store_cluster,item_cluster,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,paycycle_sin_1,paycycle_cos_1,season_sin_1,season_cos_1
0,2014-01-01,12_1012473,17,15,0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.406737,0.913545,-0.974100,0.226116
1,2014-01-02,12_1012473,17,15,0.433884,-0.900969,0.951057,0.309017,0.5,0.866025,0.743145,0.669130,-0.970064,0.242850
2,2014-01-03,12_1012473,17,15,-0.433884,-0.900969,0.951057,0.309017,0.5,0.866025,0.951057,0.309017,-0.965740,0.259512
3,2014-01-04,12_1012473,17,15,-0.974928,-0.222521,0.951057,0.309017,0.5,0.866025,0.994522,-0.104529,-0.961130,0.276097
4,2014-01-05,12_1012473,17,15,-0.781832,0.623490,0.951057,0.309017,0.5,0.866025,0.866025,-0.500000,-0.956235,0.292600
5,2014-01-06,12_1012473,17,15,0.000000,1.000000,0.951057,0.309017,0.5,0.866025,0.587785,-0.809017,-0.951056,0.309017
6,2014-01-07,12_1012473,17,15,0.781832,0.623490,0.951057,0.309017,0.5,0.866025,0.207912,-0.978148,-0.945596,0.325342
7,2014-01-08,12_1012473,17,15,0.974928,-0.222521,0.587785,-0.809017,0.5,0.866025,-0.207912,-0.978148,-0.939856,0.341571
8,2014-01-09,12_1012473,17,15,0.433884,-0.900969,0.587785,-0.809017,0.5,0.866025,-0.587785,-0.809017,-0.933837,0.357699
9,2014-01-10,12_1012473,17,15,-0.433884,-0.900969,0.587785,-0.809017,0.5,0.866025,-0.866025,-0.500000,-0.927542,0.373720


In [102]:
df[features[META_FEATURES] + features[LABELS]].head(10)

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.0,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,38.0,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,12.0,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,18.0,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,20.0,9.903488,9.903488
5,2014-01-06,12_1012473,17,15,20.0,9.903488,9.903488
6,2014-01-07,12_1012473,17,15,24.0,10.085809,10.085809
7,2014-01-08,12_1012473,17,15,23.0,10.043249,10.043249
8,2014-01-09,12_1012473,17,15,25.0,10.126631,10.126631
9,2014-01-10,12_1012473,17,15,19.0,9.852194,9.852194


In [103]:
df2[features[META_FEATURES] + features[LABELS]].head(10)

,start_date,store_item,store_cluster,item_cluster,y_sales_day_1,y_store_med_logpct_change_1,y_item_med_logpct_change_1
0,2014-01-01,12_1012473,17,15,0.000000,-6.907755,-6.907755
1,2014-01-02,12_1012473,17,15,37.999996,10.545341,10.545341
2,2014-01-03,12_1012473,17,15,11.999999,9.392662,9.392662
3,2014-01-04,12_1012473,17,15,18.000000,9.798127,9.798127
4,2014-01-05,12_1012473,17,15,20.000002,9.903487,9.903487
5,2014-01-06,12_1012473,17,15,20.000002,9.903487,9.903487
6,2014-01-07,12_1012473,17,15,24.000002,10.085809,10.085809
7,2014-01-08,12_1012473,17,15,23.000000,10.043249,10.043249
8,2014-01-09,12_1012473,17,15,24.999998,10.126631,10.126631
9,2014-01-10,12_1012473,17,15,19.000000,9.852194,9.852194
